In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#import os
os.walk('/kaggle/input')

NameError: name 'os' is not defined

In [56]:
#Convolutional Neural Network
#importing the libraries
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
tf.__version__

'2.4.1'

In [58]:
#Part 1 - Data Preprocessing
#Part 1.a - Preprocessing the Training Set
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range=0.2,zoom_range=0.2, horizontal_flip= True)
training_set = train_datagen.flow_from_directory('../input/cat-and-dog/training_set/training_set',target_size=(64, 64),batch_size= 32,class_mode="binary")

Found 8005 images belonging to 2 classes.


In [59]:
#Part 1.b - Preprocessing the Test Set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('../input/cat-and-dog/test_set/test_set',target_size= (64,64), batch_size = 32, class_mode = 'binary')

Found 2023 images belonging to 2 classes.


In [60]:
#Part 2, Building the CNN

#initialising the CNN 
cnn = tf.keras.models.Sequential()

#Part 2.a -Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32,padding = 'same', kernel_size =3, activation = 'relu', input_shape = [64,64,3]))

#Part 2.b - Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides = 2))
#Adding a second convolutional layer
cnn.add (tf.keras.layers.Conv2D(filters = 32,padding = 'same', kernel_size =3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides = 2))

#Part 2.c - Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())
#Part 2.d - Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

#Part 2.e - Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units =1, activation ='sigmoid'))

In [61]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [ ]:
#Part 3 - Training the CNN

#Part 3.a - Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])

#Part 3.b - Training the CNN on the Training Set and Evaluating it on the Test Set.
cnn.fit(x = training_set, validation_data= test_set, epochs= 15)

Epoch 1/15
 86/251 [=========>....................] - ETA: 32s - loss: 0.8277 - accuracy: 0.5160

In [14]:
#save it as a h5 file
from tensorflow.keras.models import load_model
cnn.save('model_rcat_dog.h5')

In [15]:
#load model
from tensorflow.keras.models import load_model
model = load_model('model_rcat_dog.h5')

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [53]:
#Part 4 - Making a single Prediction

import numpy as np
from tensorflow.keras.preprocessing import image

test_image = image.load_img('../input/cat-and-dog/test_set/test_set/dogs/dog.4002.jpg',target_size =(64,64))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis =0)
result = cnn.predict(test_image)

In [54]:
result[0][0]

0.38053286

In [55]:
if result[0]<=0.5:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is cat
